# 入门篇

## 1.概念

### 1.1.Python方向

讲了这么多的东西，那Python能干啥呢？这个是**我眼中的Python**：

**Python方向**：
1. 早期方向
    - **Web全栈**
2. 擅长专栏
    - **爬虫系列**
    - **数据分析**
    - **人工智能**
    - **`物联网系`**（`lot`万物互联）
    - **自动化运维**（`安全`与`测试`）
3. 其他系列
    - 游戏开发（最近很火）

如果想在Web、爬虫、物联网、游戏等等方向，网络这块是必需条件，So ==> 不要不急，咱们继续学习～

### 1.2.




## 2.UDP

### 2.1.UDP发送消息

#### 引入案例

看个UDP的简单案例：
```py
import socket

def main():

    # AF_INET ==> IPV4；SOCK_STREAM ==> 类型是TCP，stream 流
    # SOCK_DGRAM ==> 类型是UDP，dgram 数据报、数据报套接字
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as udp_sock:
        udp_sock.sendto("大兄弟，你好啊".encode("utf-8"), ("192.168.36.235", 8080))
    print("over")

if __name__ == '__main__':
    main()
```
接收到的消息：**这时候端口是随机的**
![2.UDP接收消息](https://img2018.cnblogs.com/blog/658978/201810/658978-20181030161939823-691450792.png)

看起来代码还挺麻烦，我稍微分析下你就知道对比其他语言真的太简单了：

标识：
1. `AF_INET` ==> `IPV4`
2. `SOCK_DGRAM` ==> 类型是`UDP`
3. `SOCK_STREAM` ==> 类型是`TCP`

**代码三步走**：
1. 创建 `udp_sock=socket.socket(socket.AF_INET, socket.SOCK_DGRAM)`
2. 发送 `udp_sock.sendto(Bytes内容，(IP,Port))` 接收：`udp_sock.recvfrom(count)`
3. 关闭 `udp_sock.close()`

#### 端口绑定

借助`调试工具`(<a href="https://files.cnblogs.com/files/dunitian/NetTool.tar.gz" target="_blank">点我下载</a>)可以知道：上面程序每次运行，**端口**都**不固定**
![2.UDP随机端口.png](https://img2018.cnblogs.com/blog/658978/201810/658978-20181030160106572-1670415822.png)

那怎么使用固定端口呢？==> `udp_socket.bind(('', 5400))`
```py
import socket

def main():
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as udp_socket:
        # 绑定固定端口
        udp_socket.bind(('', 5400))
        # 发送消息
        udp_socket.sendto("小明，你知道小张的生日吗？\n".encode("utf-8"),
                          ("192.168.36.235", 8080))
    print("over")

if __name__ == '__main__':
    main()
```
消息图示：`nc -ul 8080`（`nc -l`是监听TCP）
![2.nc监听UDP.png](https://img2018.cnblogs.com/blog/658978/201810/658978-20181030161609064-948308184.png)

调试工具：
![2.UDP绑定端口.png](https://img2018.cnblogs.com/blog/658978/201810/658978-20181030161114687-141010359.png)

### 2.2.UDP接收消息

先看一个简单版本的：`udp_socket.recvfrom(1024)`
```py
from socket import socket, AF_INET, SOCK_DGRAM

def main():
    with socket(AF_INET, SOCK_DGRAM) as udp_socket:
        # 绑定端口
        udp_socket.bind(('', 5400))
        while True:
            # 发送消息
            udp_socket.sendto("你可以给我离线留言了\n".encode("utf-8"),
                              ("192.168.36.235", 8080))
            # 接收消息（data,(ip,port)）
            data, info = udp_socket.recvfrom(1024)
            print(f"[来自{info[0]}:{info[1]}的消息]：\n{data.decode('utf-8')}")

if __name__ == '__main__':
    main()
```
图示：**`接收消息（data,(ip,port)）`**
![2.udp_recv.gif](https://img2018.cnblogs.com/blog/658978/201810/658978-20181030172716634-871024783.gif)

---

### 题外话（Nmap）

其实如果你使用`Nmap`来扫描的话并不能发现`nc`打开的`UDP`端口：
![2.nmap的UDP扫描.gif](https://img2018.cnblogs.com/blog/658978/201810/658978-20181030172711902-1875073285.gif)

稍微解释一下：**扫描其实就是发了几个空消息过去**
1. `-sU`代表扫描UDP，`-sT`代表扫描TCP
2. `-Pn` 这个主要是针对有些服务器禁用ping的处理（ping不通也尝试）
3. `-p` 指定端口号，如果是所有端口可以使用`-p-`
4. `sudo`是因为在`Ubuntu`下没权限，`kali`下可以直接使用`nmap`

可能有人对`nc`输出的*`你可以给离线留意了`*有疑惑，其实就是在给5400端口发空消息的时候～`True循环了两次`

来张对比图：
![2.nc找不到.gif](https://img2018.cnblogs.com/blog/658978/201810/658978-20181031093925016-1829175408.gif)

**扫描TCP和UDP端口**：`sudo nmap -sTU 192.168.36.235 -Pn`

**课后扩展**：
```
NC命令扩展：https://www.cnblogs.com/nmap/p/6148306.html

Nmap基础：https://www.cnblogs.com/dunitian/p/5074784.html
```

#### 收放自如

如果还是用True循环来实现：
```py
from socket import socket, AF_INET, SOCK_DGRAM

def main():
    with socket(AF_INET, SOCK_DGRAM) as udp_socket:
        # 绑定端口
        udp_socket.bind(('', 5400))
        while True:
            msg = input("请输入发送的内容：")
            if msg == "dotnetcrazy":
                break
            else:
                udp_socket.sendto(
                    msg.encode("utf-8"), ("192.168.36.235", 8080))

            data, info = udp_socket.recvfrom(1024)
            print(f"[来自{info[0]}:{info[1]}的消息]：\n{data.decode('utf-8')}")

if __name__ == '__main__':
    main()
```
你会发现，消息不能轮流发送，只能等对方方式后再发，虽然有处理方式，但太麻烦，这时候就可以使用我们之前说的**多线程来改写**一下了：
```py
from socket import socket, AF_INET, SOCK_DGRAM
from multiprocessing.dummy import Pool as ThreadPool

def send_msg(udp_socket):
    while True:
        msg = input("输入需要发送的消息：\n")
        udp_socket.sendto(msg.encode("utf-8"), ("192.168.36.235", 8080))

def recv_msg(udp_socket):
    while True:
        data, info = udp_socket.recvfrom(1024)
        print(f"[来自{info[0]}:{info[1]}的消息]：\n{data.decode('utf-8')}")

def main():
    # 创建一个Socket
    with socket(AF_INET, SOCK_DGRAM) as udp_socket:
        # 绑定端口
        udp_socket.bind(('', 5400))

        # 创建一个线程池
        pool = ThreadPool()

        # 接收消息
        pool.apply_async(recv_msg, args=(udp_socket, ))

        # 发送消息
        pool.apply_async(send_msg, args=(udp_socket, ))

        pool.close()  # 不再添加任务
        pool.join()  # 等待线程池执行完毕
    print("over")

if __name__ == '__main__':
    main()
```
输出：（就一个注意点～`socket在pool之后关闭`）
![2.收放自如.gif](https://img2018.cnblogs.com/blog/658978/201810/658978-20181031084637702-678648461.gif)

---

### 2.3.手写UDP网络调试工具

调试工具功能比较简单，我们手写一个`UDP`版的：
```py
from socket import socket, AF_INET, SOCK_DGRAM
from multiprocessing.dummy import Pool as ThreadPool

def get_port(msg):
    """获取用户输入的端口号"""
    while True:
        port = input(msg)
        try:
            port = int(port)
        except Exception as ex:
            print(ex)
        else:
            return port  # 没有错误就退出死循环

def recv_msg(udp_socket):
    """接收消息"""
    while True:
        data, info = udp_socket.recvfrom(1024)
        print(f"[来自{info[0]}:{info[1]}的消息]：\n{data.decode('utf-8')}")

def send_msg(udp_socket):
    """发送消息"""
    ip = input("请输入对方IP：")
    port = get_port("请输入对方端口号：")
    while True:
        msg = input("请输入发送的消息：\n")
        udp_socket.sendto(msg.encode("utf-8"), (ip, port))

def main():
    with socket(AF_INET, SOCK_DGRAM) as udp_socket:
        # 绑定端口
        udp_socket.bind(('', get_port("请输网络助手的端口号：")))
        # 创建一个线程池
        pool = ThreadPool()
        # 接收消息
        pool.apply_async(recv_msg, args=(udp_socket, ))
        # 发送消息
        pool.apply_async(send_msg, args=(udp_socket, ))

        pool.close()
        pool.join()

if __name__ == '__main__':
    main()

```

CentOS`IP`和`Port`(`192.168.36.123:5400`)
![2.UDP网络助手.png](https://img2018.cnblogs.com/blog/658978/201810/658978-20181031130707093-968035575.png)

演示：(**多PC演示**)
![2.UDPTool.gif](https://img2018.cnblogs.com/blog/658978/201810/658978-20181031131353347-509340602.gif)

简单说下本机IP的绑定：

Net里面习惯使用`localhost`，很多人不知道到底是啥，其实你打开`host`文件就可以看到 ==> `127.0.0.1`被重定向为`localhost`，在Linux里面也是这样的，每个PC对应的都是`lo`回环地址：
![2.lo.png](https://img2018.cnblogs.com/blog/658978/201810/658978-20181031132010394-1833915775.png)

**本机通信时，对方ip就可以使用`127.0.0.1`了，当然了绑定本机ip的时候也可以使用`127.0.0.1`（`bind(('',))`中的空其实填的就是这个）**(很多地方也会使用`0.0.0.0`)
```py
_LOCALHOST    = '127.0.0.1' # 看这
_LOCALHOST_V6 = '::1'

   def socketpair(family=AF_INET, type=SOCK_STREAM, proto=0):
        if family == AF_INET:
            host = _LOCALHOST # 看这
        elif family == AF_INET6:
            host = _LOCALHOST_V6
        ....
        
        lsock = socket(family, type, proto)
        try:
            lsock.bind((host, 0)) # 看这
            lsock.listen()
            ...
```

## 3.TCP

用过下载软件的可能遇到过一种`‘Bug’` ==> 很多人为了防止自己本地文件纳入共享大军，一般都是直接把网络上传给禁了，然后发现文件经常出问题？

其实这个就是`TCP`的一个应用，文件一般都很大，所以进行分割后批量下载，那少量的网络上传其实是为了校验一下文件 ==> 正确做法是限制上传速度而不是禁止（学生时代那会还经常蛋疼这个问题，现在想想还挺好玩的`O(∩_∩)O`）

大多数连接都是可靠的TCP连接。**创建TCP连接时，主动发起连接的叫客户端，被动响应连接的叫服务器**

上面那个例子里，我们的下载工具就是客户端，每一小段文件接收完毕后都会向服务器发送一个完成的指令来保证文件的完整性

### 3.1.TCP客户端

来看一个简单的入门案例：
```py
from socket import socket

def main():
    # 默认就是创建TCP Socket
    with socket() as tcp_socket:
        # 连接服务器（没有返回值）
        tcp_socket.connect(("192.168.36.235", 8080))
        # 发送消息（返回发送的字节数）
        tcp_socket.send("小张生日快乐～".encode("utf-8"))
        # 接收消息
        msg = tcp_socket.recv(1024)
        print(f"服务器：{msg.decode('utf-8')}")

if __name__ == '__main__':
    main()
```
输出：（`socket()`默认就是创建`TCP Socket`）
![3.tcp_client.gif](https://img2018.cnblogs.com/blog/1127869/201811/1127869-20181101101153493-941765705.gif)

概括来说：
1. **TCP，有点像打电话，先拨号连通了(`connect`)才能通信(`send`，`recv`)，之后的通信不用再拨号连通了**
2. **UDP，有点像寄信封，每次寄过去都不确定能不能收到，每次通信都得写地址(`ip`+`port`)**

**代码四步走**：（TCP客户端其实创建`Socket`之后`connect`一下服务器就OK了）
1. 创建：`tcp_sock=socket.socket(socket.AF_INET, socket.SOCK_STREAM)`
2. 连接：`tcp_sock.connect((IP, Port))`
3. 发送：`tcp_sock.send(Bytes内容)` 接收：`tcp_sock.recv(count)`
4. 关闭：`tcp_sock.close()`

#### 模拟HTTP
```py
from socket import socket

def get_buffer(tcp_socket):
    buffer_list = []
    while True:
        b = tcp_socket.recv(1024)
        if b:
            buffer_list.append(b)
        else:
            break
    # 拼接在一起
    return b''.join(buffer_list)

def main():
    with socket() as tcp_socket:
        # 连接服务器
        tcp_socket.connect(("dotnetcrazy.cnblogs.com", 80))
        # 发送消息（模拟HTTP）
        tcp_socket.send(
            b'GET / HTTP/1.1\r\nHost: dotnetcrazy.cnblogs.com\r\nConnection: close\r\n\r\n'
        )
        # 以"\r\n\r\n"分割一次
        header, data = get_buffer(tcp_socket).split(b"\r\n\r\n", 1)
        print(header.decode("utf-8"))
        with open("test.html", "wb") as f:
            f.write(data)
    print("over")

if __name__ == '__main__':
    main()
```
输出：（`test.html`就是页面源码）
```
HTTP/1.1 200 OK
Date: Thu, 01 Nov 2018 03:10:48 GMT
Content-Type: text/html; charset=utf-8
Content-Length: 20059
Connection: close
Vary: Accept-Encoding
Cache-Control: private, max-age=10
Expires: Thu, 01 Nov 2018 03:10:58 GMT
Last-Modified: Thu, 01 Nov 2018 03:10:48 GMT
X-UA-Compatible: IE=10
X-Frame-Options: SAMEORIGIN
over
```
**注意`\r\n`和`Connection:close`；`split("",分割次数)`**

---

### 3.2.TCP服务端

服务端代码相比于UDP，多了一个监听和等待客户端，其他基本上一样：

客户端Code：(如果你想固定端口也可以绑定一下`Port`)
```py
from socket import socket

def main():
    # 默认就是创建TCP Socket
    with socket() as tcp_socket:
        # 连接服务器（没有返回值）
        tcp_socket.connect(("192.168.36.235", 8080))

        print("Connected TCP Server...")  # 连接提示

        # 发送消息（返回发送的字节数）
        tcp_socket.send("小张生日快乐～\n".encode("utf-8"))
        # 接收消息
        msg = tcp_socket.recv(1024)
        print(f"服务器：{msg.decode('utf-8')}")

if __name__ == '__main__':
    main()
```
服务端Code：
```py
from socket import socket

def main():
    with socket() as tcp_socket:
        # 绑定端口（便于客户端找到）
        tcp_socket.bind(('', 8080))
        # 变成被动接收消息（监听）
        tcp_socket.listen()  # 不指定连接最大数则会设置默认值

        print("TCP Server is Running...")  # 运行后提示

        # 等待客户端发信息
        client_socket, client_addr = tcp_socket.accept()

        with client_socket:
            # 客户端连接提示
            print(f"[来自{client_addr[0]}:{client_addr[1]}的消息]\n")

            # 接收客户端消息
            data = client_socket.recv(1024)
            print(data.decode("utf-8"))

            # 回复客户端
            client_socket.send("知道了".encode("utf-8"))

if __name__ == '__main__':
    main()
```
输出：（先运行服务端，再运行客户端。客户端发了一个生日快乐的祝福，服务端回复了一句）
![3.tcp_server.gif](https://img2018.cnblogs.com/blog/1127869/201811/1127869-20181101152713008-441395206.gif)

### 3.2.TCP服务端调试助手

如果像上面那般，并不能多客户端通信
![3.bug.png](https://img2018.cnblogs.com/blog/1127869/201811/1127869-20181101171545763-1800450636.png)

这时候可以稍微改造一下：

#### 客户端：

```py
from time import sleep
from socket import socket
from multiprocessing.dummy import Pool

def send_msg(tcp_socket):
    with tcp_socket:
        while True:
            try:
                tcp_socket.send("小明同志\n".encode("utf-8"))
                sleep(2)  # send是非阻塞的
                print("向服务器问候了一下")
            except Exception as ex:
                print("服务端连接已断开:", ex)
                break

def recv_msg(tcp_socket):
    with tcp_socket:
        while True:
            # 这边可以不捕获异常：
            #    服务端关闭时，send_msg会关闭，然后这边也就关闭了
            try:
                data = tcp_socket.recv(1024)
                if data:
                    print("服务端回复：", data.decode("utf-8"))
            except Exception as ex:
                print("tcp_socket已断开:", ex)
                break

def main():
    with socket() as tcp_socket:
        # 连接TCP Server
        tcp_socket.connect(("192.168.36.235", 8080))
        print("Connected TCP Server...")  # 连接提示

        pool = Pool()
        pool.apply_async(send_msg, args=(tcp_socket,))
        pool.apply_async(recv_msg, args=(tcp_socket,))
        pool.close()
        pool.join()

if __name__ == '__main__':
    main()
```
#### 服务端

**服务器需要同时响应多个客户端的请求，那么每个连接都需要一个新的进程或者线程来处理**

```py
from socket import socket
from multiprocessing.dummy import Pool

def wait_client(client_socket, ip_port):
    with client_socket:
        while True:
            data = client_socket.recv(1024)
            print(f"[来自{ip_port}的消息]:\n{data.decode('utf-8')}")
            client_socket.send(b"I Know")  # bytes类型

def main():
    with socket() as tcp_socket:
        # 绑定端口
        tcp_socket.bind(('', 8080))
        # 服务器监听
        tcp_socket.listen()

        print("TCP Server is Running...")  # 运行后提示

        p = Pool()
        while True:
            # 等待客户端连接
            client_socket, client_addr = tcp_socket.accept()
            ip_port = f"{client_addr[0]}:{client_addr[1]}"
            print(f"客户端{ip_port}已连接")
            # 响应多个客户端则需要多个线程来处理
            p.apply_async(wait_client, args=(client_socket, ip_port))

if __name__ == '__main__':
    main()
```
演示：（死循环，`Pool`都不用管了）
![3.正常流程.gif](https://img2018.cnblogs.com/blog/1127869/201811/1127869-20181102170246044-731640129.gif)

服务器挂了客户端也会自动退出：
![3.自动退出.gif](https://img2018.cnblogs.com/blog/1127869/201811/1127869-20181102170240157-2015402018.gif)

用TCP协议进行`Socket`编程在Python中十分简单：
1. 客户端：主动连接服务器的IP和指定端口
2. 服务器：先监听指定端口，然后对每一个新的连接创建一个线程或进程来处理

---

### 扩展

上面忘记说了，`Socket`是可以设置超时时间的，eg：`tcp_socket.settimeout(3)`

#### 探一探`localhost`

代码不变，如果把TCP客户端的`连接服务器IP`空着或者改成`127.0.0.1`，咱们再看看效果：`tcp_socket.connect(('', 8080))`

图示：（怎么样，这回知道本机问啥可以不写IP了吧）
![3.localhost.png](https://img2018.cnblogs.com/blog/1127869/201811/1127869-20181101153332389-592687559.png)

#### 手写一个端口扫描工具

端口扫描大家不陌生，自己实现一个简单的TCP端口扫描工具：
```py
from socket import socket
from multiprocessing.dummy import Pool

ip = "127.0.0.1"

def tcp_port(port):
    """IP：服务端IP，Port：服务端Port"""
    with socket() as tcp_socket:
        try:
            tcp_socket.connect((ip, port))
            print(f"[TCP Port:{port} is open]")
        except Exception:
            pass

def main():
    # 查看系统本地可用端口极限值 cat /proc/sys/net/ipv4/ip_local_port_range
    max_port = 60999
    global ip
    ip = input("请输入要扫描的IP地址：")
    print(f"正在对IP：{ip}进行端口扫描...")

    pool = Pool()
    pool.map_async(tcp_port, range(max_port))
    pool.close()
    pool.join()

if __name__ == '__main__':
    main()
```
输出：
```
dnt@MZY-PC:~/桌面/work/BaseCode/python/6.net/3.Ext python3 1.port_scan.py 
请输入要扫描的IP地址：192.168.36.235
正在对IP：192.168.36.235进行端口扫描...
[TCP Port:22 is open]
[TCP Port:41004 is open]
dnt@MZY-PC:~/桌面/work/BaseCode/python/6.net/3.Ext sudo nmap -sT 192.168.36.235 -Pn -p-

Starting Nmap 7.60 ( https://nmap.org ) at 2018-11-02 18:15 CST
Nmap scan report for MZY-PC (192.168.36.235)
Host is up (0.000086s latency).
Not shown: 65534 closed ports
PORT   STATE SERVICE
22/tcp open  ssh

Nmap done: 1 IP address (1 host up) scanned in 2.07 seconds
```

#### 课后思考

课后思考：**为啥发送(`send`、`sendto`)和接收(`recv`、`recvfrom`)都是两个方法？**（提示：`方法名`、`阻塞`）

课外拓展：
```
TCP编程知识
https://dwz.cn/dDkXzqcV

网络编程-基础
https://www.jianshu.com/p/55c171ebe5f1

网络编程-UDP
https://www.jianshu.com/p/594870b1634b

网络编程-TCP
https://www.jianshu.com/p/be36d4db5618

Python总结之 recv与recv_from
https://www.jianshu.com/p/5643e810123f
https://blog.csdn.net/xvd217/article/details/38902081
https://blog.csdn.net/pengluer/article/details/8812333

Python socket借助ngrok建立外网TCP连接
https://www.jianshu.com/p/913b2013a38f
```

# 加强篇

# 深入篇